In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")

In [3]:
from deepsvg.svglib.svg import SVG

from deepsvg import utils
from deepsvg.difflib.tensor import SVGTensor
from deepsvg.svglib.utils import to_gif
from deepsvg.svglib.geom import Bbox
from deepsvg.svgtensor_dataset import SVGTensorDataset, load_dataset
from deepsvg.utils.utils import batchify, linear

import torch
import numpy as np

# DeepSVG latent space operations

In [4]:
device = torch.device("cuda:0"if torch.cuda.is_available() else "cpu") 

Load the pretrained model and dataset

In [5]:
pretrained_path = "./pretrained/hierarchical_ordered.pth.tar"
from configs.deepsvg.hierarchical_ordered import Config

try:
    state_dict = torch.load(pretrained_path, map_location=torch.device('cpu'))  # Use 'cuda' if you have GPU
    print("File loaded successfully.")
    print(state_dict.keys())  # Prints the keys to verify the content
except Exception as e:
    print(f"Error loading the file: {e}")

cfg = Config()
model = cfg.make_model().to(device)
utils.load_model(pretrained_path, model)
model.eval();

File loaded successfully.
dict_keys(['model'])


In [6]:
dataset = load_dataset(cfg)

In [7]:
def load_svg(filename):
    svg = SVG.load_svg(filename)
    #svg = dataset.simplify(svg)
    #svg = dataset.preprocess(svg, augment=False)
    svg.canonicalize()
    svg.normalize()
    svg.zoom(0.9)
    svg = svg.simplify_heuristic()
    svg =svg.numericalize(256)
    return svg

In [8]:
def encode(data):
    model_args = batchify((data[key] for key in cfg.model_args), device)
    with torch.no_grad():
        z = model(*model_args, encode_mode=True)
        return z

def encode_icon(idx):
    data = dataset.get(id=idx, random_aug=False)
    return encode(data)
    
def encode_svg(svg):
    data = dataset.get(svg=svg)
    return encode(data)

def decode(z, do_display=True, return_svg=False, return_png=False):
    commands_y, args_y = model.greedy_sample(z=z)
    tensor_pred = SVGTensor.from_cmd_args(commands_y[0].cpu(), args_y[0].cpu())
    svg_path_sample = SVG.from_tensor(tensor_pred.data, viewbox=Bbox(256), allow_empty=True).normalize().split_paths().set_color("random")
    
    if return_svg:
        return svg_path_sample
    
    return svg_path_sample.draw(do_display=do_display, return_png=return_png)

In [9]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask import send_from_directory

import os
import uuid
from PIL import Image
import cairosvg

import numpy as np
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
import cairosvg
from IPython.display import display
from PIL import Image

DeepSVG

In [10]:
app = Flask(__name__)
CORS(app) 

DATASET_DIR = "./dataset/Registered_Dataset_simplified"
PNG_OUTPUT_DIR = "./dataset/rendered_pngs"

def load_and_encode(svg_path):
    try:
        svg = load_svg(svg_path)
        vector = encode_svg(svg)
        return vector
    except Exception as e:
        print(f"Encoding failed for {svg_path}: {e}")
        return None

@app.route('/api/lookup-logo', methods=['POST'])
def lookup_logo():
    if 'logo' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['logo']
    if not file or file.filename == '':
        return jsonify({'error': 'Invalid file'}), 400

    if not file.filename.endswith('.svg'):
        return jsonify({'error': 'Only SVG files are allowed'}), 400

    # Save uploaded SVG temporarily
    temp_id = str(uuid.uuid4())
    temp_path = f"./temp/{temp_id}.svg"
    os.makedirs("./temp", exist_ok=True)
    file.save(temp_path)

    target_vector = load_and_encode(temp_path)
    if target_vector is None:
        return jsonify({'error': 'SVG encoding failed'}), 500

    similarities = []
    for fname in os.listdir(DATASET_DIR):
        if not fname.endswith('.svg'):
            continue
        candidate_path = os.path.join(DATASET_DIR, fname)
        candidate_vector = load_and_encode(candidate_path)
        if candidate_vector is None:
            continue
        similarity = 1 - cosine(target_vector, candidate_vector)
        similarities.append((fname, similarity, candidate_path))

    similarities.sort(key=lambda x: x[1], reverse=True)
    top_matches = similarities[:3]

    results = []
    for fname, score, path in top_matches:
        # Convert to PNG
        png_name = fname.replace('.svg', '.png')
        png_path = os.path.join(PNG_OUTPUT_DIR, png_name)
        if not os.path.exists(png_path):
            try:
                cairosvg.svg2png(file_obj=open(path, "rb"), write_to=png_path)
            except:
                continue

        results.append({
            "logoUrl": f"http://localhost:5000/static/{png_name}",
            "companyUrl": f"https://example.com/brand/{fname.replace('.svg','')}"  # Replace with real metadata
        })

    os.remove(temp_path)  # Clean up
    return jsonify({'matches': results})

# Serve static images
@app.route('/static/<filename>')
def serve_png(filename):
    return send_from_directory(PNG_OUTPUT_DIR, filename)

app.run(port=5000, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Apr/2025 09:35:56] "POST /api/lookup-logo HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 09:35:56] "GET /static/303110_apple-black-logo.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Apr/2025 09:35:56] "GET /static/303238_google-drive-logo.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Apr/2025 09:35:56] "GET /static/chrome-color-svgrepo-com.png HTTP/1.1" 200 -


Algorithma based methode- procrustes

In [19]:
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import numpy as np
import uuid
import cairosvg
import base64
from scipy.spatial import procrustes
from svgpathtools import svg2paths, Path

app = Flask(__name__)
CORS(app)

DATASET_DIR = "./dataset/Registered_Dataset_simplified"
PNG_OUTPUT_DIR = "./dataset/rendered_pngs"

os.makedirs(PNG_OUTPUT_DIR, exist_ok=True)

# ---- Helper Functions ---- #

def join_svg_paths(svg_file):
    paths, _ = svg2paths(svg_file)
    combined_path = Path()
    for path in paths:
        combined_path.extend(path)
    return combined_path

def parse_svg(svg_path, num_samples=250):
    path = join_svg_paths(svg_path)
    total_length = path.length()
    sample_distances = np.linspace(0, total_length, num_samples)
    sampled_points = []
    for distance in sample_distances:
        point = path.point(distance / total_length)
        sampled_points.append((point.real, point.imag))
    return np.array(sampled_points)

def load_and_encode(svg_path):
    try:
        return parse_svg(svg_path)
    except Exception as e:
        print(f"Encoding failed for {svg_path}: {e}")
        return None

def compute_procrustes_similarity(shape1, shape2):
    try:
        _, _, disparity = procrustes(shape1, shape2)
        return 1 / (1 + disparity)
    except Exception as e:
        print(f"Procrustes comparison failed: {e}")
        return 0

# ---- API Endpoint ---- #

@app.route('/api/lookup-logo', methods=['POST'])
def lookup_logo():
    if 'logo' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['logo']
    if not file or file.filename == '' or not file.filename.endswith('.svg'):
        return jsonify({'error': 'Only SVG files are allowed'}), 400

    temp_id = str(uuid.uuid4())
    temp_path = f"./temp/{temp_id}.svg"
    os.makedirs("./temp", exist_ok=True)
    file.save(temp_path)

    target_vector = load_and_encode(temp_path)
    if target_vector is None:
        os.remove(temp_path)
        return jsonify({'error': 'SVG encoding failed'}), 500

    similarities = []
    for fname in os.listdir(DATASET_DIR):
        if not fname.endswith('.svg'):
            continue
        candidate_path = os.path.join(DATASET_DIR, fname)
        candidate_vector = load_and_encode(candidate_path)
        if candidate_vector is None:
            continue
        score = compute_procrustes_similarity(target_vector, candidate_vector)
        similarities.append((fname, score, candidate_path))

    similarities.sort(key=lambda x: x[1], reverse=True)
    top_matches = similarities[:3]

    results = []
    for fname, score, path in top_matches:
        png_name = fname.replace('.svg', '.png')
        png_path = os.path.join(PNG_OUTPUT_DIR, png_name)
        if not os.path.exists(png_path):
            try:
                cairosvg.svg2png(file_obj=open(path, "rb"), write_to=png_path)
            except Exception as e:
                print(f"PNG conversion failed for {fname}: {e}")
                continue

        results.append({
            "logoUrl": f"http://localhost:5000/static/{png_name}",
            "companyUrl": f"https://example.com/brand/{fname.replace('.svg','')}",
            "score": round(score, 4)
        })

    os.remove(temp_path)
    return jsonify({'matches': results})

# ---- Serve PNGs ---- #

@app.route('/static/<filename>')
def serve_png(filename):
    return send_from_directory(PNG_OUTPUT_DIR, filename)

app.run(port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Apr/2025 07:31:34] "POST /api/lookup-logo HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 07:31:34] "GET /static/303190_after-effects-cc-logo.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Apr/2025 07:31:34] "GET /static/303296_siri-apple-logo.png HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 07:31:34] "GET /static/303184_adobe-illustrator-cc-logo.png HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 07:31:48] "POST /api/lookup-logo HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 07:31:48] "GET /static/303117_facebook-2-logo.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Apr/2025 07:31:48] "GET /static/303110_apple-black-logo.png HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 07:31:48] "GET /static/303303_oracle-6-logo.png HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 07:32:06] "POST /api/lookup-logo HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 07:32:07] "GET /static/303145_instagram-2-1-logo.png HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 07:32:07] "GET /sta

CDAF

In [21]:
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import numpy as np
import uuid
import cairosvg
from scipy.spatial.distance import euclidean
from svgpathtools import svg2paths, Path
from PIL import Image as ImagePil
import matplotlib.pyplot as plt
import io

app = Flask(__name__)
CORS(app)

DATASET_DIR = "./dataset/Registered_Dataset_simplified"
PNG_OUTPUT_DIR = "./dataset/rendered_pngs"
os.makedirs(PNG_OUTPUT_DIR, exist_ok=True)

# ---- Helper Functions ---- #

def join_svg_paths(svg_file):
    paths, _ = svg2paths(svg_file)
    combined_path = Path()
    for path in paths:
        combined_path.extend(path)
    return combined_path

def parse_svg(svg_path, num_samples=200):
    path = join_svg_paths(svg_path)
    total_length = path.length()
    sample_distances = np.linspace(0, total_length, num_samples)
    sampled_points = []
    for distance in sample_distances:
        point = path.point(distance / total_length)
        sampled_points.append((point.real, point.imag))
    return np.array(sampled_points)

def ensure_consistent_direction(points):
    s = sum(
        points[i, 0] * points[(i + 1) % len(points), 1] - points[(i + 1) % len(points), 0] * points[i, 1]
        for i in range(len(points))
    )
    if s < 0:
        points = points[::-1]
    return points

def calculate_centroid(points):
    return np.mean(points[:, 0]), np.mean(points[:, 1])

def calculate_features(points, centroid):
    gx, gy = centroid
    r = np.sqrt((points[:, 0] - gx) ** 2 + (points[:, 1] - gy) ** 2)
    theta = []
    for i, (x, y) in enumerate(points):
        tangent = points[(i + 1) % len(points)] - points[i - 1]
        dx, dy = x - gx, y - gy
        angle = abs(np.arctan2(dy, dx) - np.arctan2(tangent[1], tangent[0]))
        theta.append(min(angle, np.pi / 2))
    return r / np.max(r), np.array(theta)

def calculate_similarity(r1, theta1, r2, theta2):
    r_similarity = 1 - euclidean(r1, r2) / np.sqrt(len(r1))
    theta_similarity = np.dot(theta1, theta2) / (np.linalg.norm(theta1) * np.linalg.norm(theta2))
    return r_similarity + theta_similarity

def calculate_max_similarity(r1, theta1, r2, theta2):
    max_similarity = 0
    n = len(r1)
    for shift in range(n):
        r2_shifted = np.roll(r2, shift)
        theta2_shifted = np.roll(theta2, shift)
        sim = calculate_similarity(r1, theta1, r2_shifted, theta2_shifted)
        max_similarity = max(max_similarity, sim)
    return max_similarity

def svg_shape_similarity(query_svg_path, dataset_dir):
    query_points = parse_svg(query_svg_path)
    query_points = ensure_consistent_direction(query_points)
    q_centroid = calculate_centroid(query_points)
    q_r, q_theta = calculate_features(query_points, q_centroid)

    similarities = []
    for fname in os.listdir(dataset_dir):
        if not fname.endswith('.svg'):
            continue
        candidate_path = os.path.join(dataset_dir, fname)
        try:
            target_points = parse_svg(candidate_path)
            target_points = ensure_consistent_direction(target_points)
            t_centroid = calculate_centroid(target_points)
            t_r, t_theta = calculate_features(target_points, t_centroid)
            score = calculate_max_similarity(q_r, q_theta, t_r, t_theta)
            similarities.append((fname, score, candidate_path))
        except Exception as e:
            print(f"Error processing {fname}: {e}")
            continue

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:3]

def convert_svg_to_png(svg_path, png_path):
    try:
        cairosvg.svg2png(file_obj=open(svg_path, "rb"), write_to=png_path)
    except Exception as e:
        print(f"PNG conversion failed for {svg_path}: {e}")

# ---- API Endpoint ---- #

@app.route('/api/lookup-logo', methods=['POST'])
def lookup_logo():
    if 'logo' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['logo']
    if not file or file.filename == '' or not file.filename.endswith('.svg'):
        return jsonify({'error': 'Only SVG files are allowed'}), 400

    temp_id = str(uuid.uuid4())
    temp_path = f"./temp/{temp_id}.svg"
    os.makedirs("./temp", exist_ok=True)
    file.save(temp_path)

    try:
        results = svg_shape_similarity(temp_path, DATASET_DIR)
    except Exception as e:
        os.remove(temp_path)
        return jsonify({'error': f"Error processing SVG: {str(e)}"}), 500

    matches = []
    for fname, score, svg_path in results:
        png_name = fname.replace('.svg', '.png')
        png_path = os.path.join(PNG_OUTPUT_DIR, png_name)
        if not os.path.exists(png_path):
            convert_svg_to_png(svg_path, png_path)

        matches.append({
            "logoUrl": f"http://localhost:5000/static/{png_name}",
            "companyUrl": f"https://example.com/brand/{fname.replace('.svg','')}",
            "score": round(score, 4)
        })

    os.remove(temp_path)
    return jsonify({'matches': matches})

# ---- Serve PNGs ---- #

@app.route('/static/<filename>')
def serve_png(filename):
    return send_from_directory(PNG_OUTPUT_DIR, filename)

app.run(port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Apr/2025 08:00:29] "POST /api/lookup-logo HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 08:00:29] "GET /static/303145_instagram-2-1-logo.png HTTP/1.1" 304 -
127.0.0.1 - - [06/Apr/2025 08:00:29] "GET /static/303177_photoshop-cc-logo.png.png HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2025 08:00:29] "GET /static/303177_photoshop-cc-logo.png HTTP/1.1" 200 -
